In [1]:
from plugins.environments.awa_environment import AWAEnvironment
from plugins.interfaces.awa_interface import AWAInterface

# import data from csv file
import pandas as pd
variable_file = "plugins/environments/awa_variables.csv"
observable_file = "plugins/environments/awa_observables.csv"

env = AWAEnvironment(variable_file, observable_file, interface=AWAInterface(), target_charge=1.0)

In [2]:
env.variables

{'AWA:Drive:DS1:Ctrl': [500.0, 550.0],
 'AWA:Drive:DS3:Ctrl': [180.0, 260.0],
 'AWA:Bira3Ctrl:Ch03': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch04': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch05': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch06': [-5.0, 5.0],
 'AWALLRF:K1:SetPhase': [236.0, 256.0],
 'AWA:DAC0:Ch08': [4.5, 5.9]}

In [3]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.bayesian import BayesianExplorationGenerator, UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from xopt.generators.bayesian.turbo import SafetyTurboController, OptimizeTurboController

import time

def evaluate(inputs):
    env.set_variables(inputs)
    time.sleep(2.0)
    result = env.get_observables(["13ARV1:Sx"])
    result["total_rms_size"] = (result["13ARV1:Sx"]**2 + result["13ARV1:Sy"]**2)**0.5
    return result

# use only one variable
vocs = VOCS(variables=env.variables,
            objectives={"total_rms_size":"MINIMIZE"},
            constraints={
                "13ARV1:bb_penalty":["LESS_THAN",0.0],
                "13ARV1:log10_total_intensity":["GREATER_THAN", 4.5]
            })


In [4]:

model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = BayesianExplorationGenerator(
    vocs=vocs, 
    model_constructor=model_constructor,
    turbo_controller=SafetyTurboController(vocs=vocs, length=0.1)
)
evaluator = Evaluator(function=evaluate)
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)
X.options.dump_file = "exploration_6_nd_filter.yml"

##### X

In [5]:
# get the current quad setpoint
#current_val = env.get_variables(['AWA:Bira3Ctrl:Ch03'])
import numpy as np
default_pt = np.array([0, 0, 0, 0,5.9,550,190,246])
default_val = dict(zip(X.vocs.variable_names, default_pt))
default_val = pd.DataFrame(default_val, index=[0])
default_val

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase
0,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0


In [6]:
# evaluate that point in xopt
X.evaluate_data(default_val)

"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 16:33:32.399469443
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize1_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 16:33:32.399378796
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize0_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArrayData", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 16:33:32.399547309
................................................................

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,AWAVXI11ICT:Ch1,13ARV1:image1:ArraySize1_RBV,...,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:bb_penalty_std,13ARV1:total_intensity_std,13ARV1:log10_total_intensity_std,total_rms_size,xopt_runtime,xopt_error
1,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0,9.383756e-10,1200.0,...,1.065201,0.640701,2.305501,1.321199,5.914229,147542.155452,0.041311,58.781718,9.21672,False


In [7]:
# evaluate a second nearby point
second_pt = np.array([0.1, 0.1, 0.1, 0.1, 5.8,540,185,245])
second_pt = dict(zip(X.vocs.variable_names, second_pt))
second_pt = pd.DataFrame(second_pt, index=[0])
X.evaluate_data(second_pt)

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,AWAVXI11ICT:Ch1,13ARV1:image1:ArraySize1_RBV,...,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:bb_penalty_std,13ARV1:total_intensity_std,13ARV1:log10_total_intensity_std,total_rms_size,xopt_runtime,xopt_error
2,0.1,0.1,0.1,0.1,5.8,540.0,185.0,245.0,9.522137e-10,1200.0,...,0.830011,0.468287,5.27084,0.196187,10.956383,209345.817651,0.049094,101.925278,12.218099,False


In [8]:
X.data

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,AWAVXI11ICT:Ch1,13ARV1:image1:ArraySize1_RBV,...,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:bb_penalty_std,13ARV1:total_intensity_std,13ARV1:log10_total_intensity_std,total_rms_size,xopt_runtime,xopt_error
1,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0,9.383756e-10,1200.0,...,1.065201,0.640701,2.305501,1.321199,5.914229,147542.155452,0.041311,58.781718,9.216720,False
2,0.1,0.1,0.1,0.1,5.8,540.0,185.0,245.0,9.522137e-10,1200.0,...,0.830011,0.468287,5.270840,0.196187,10.956383,209345.817651,0.049094,101.925278,12.218099,False


In [9]:
# run exploration
n_steps = 200
X.generator.numerical_optimizer.max_iter = 100
for i in range(n_steps):
    print(i)
    start = time.time()
    X.step()
    print(time.time() - start)
    

0
21.868786573410034
1
16.021493673324585
2
18.9787437915802
3
13.02622389793396
4
16.044309377670288
5
10.941648244857788
6
12.519606828689575
7
16.511353015899658
8
19.48990797996521
9
11.011896848678589
10
17.98816990852356
11
12.813816785812378
12
15.713447093963623
13
16.508463621139526
14
10.270737409591675
15
12.478586435317993
16
13.023021221160889
17
15.740731477737427
18
15.775237798690796
19
15.508583784103394
20
22.501044511795044
21
17.735605239868164
22
16.504708528518677
23
12.492563724517822
24
11.771834135055542
25
13.084902048110962
26
13.68036699295044
27
14.015722274780273
28
13.481832027435303
29
17.036664724349976
30
12.547338485717773
31
13.454883337020874
32
14.331367492675781
33


UnboundLocalError: local variable 'result' referenced before assignment

In [ ]:
X.data

In [ ]:
X.data.plot(y=X.vocs.variable_names[:4])

In [ ]:
X.data.plot(y=X.vocs.variable_names[4])

In [ ]:
X.data.plot(y=X.vocs.variable_names[5:])

In [ ]:
import torch
import matplotlib.pyplot as plt
test_x = torch.linspace(-2,2, 100)
model = X.generator.train_model()

fig,ax = plt.subplots(2,1, sharex="all")
fig.set_size_inches(6,6)
with torch.no_grad():
    post = model.posterior(test_x.reshape(-1,1,1).double())
    for i in range(post.event_shape[-1]):
        mean = post.mean[...,i].squeeze()
        l,u = post.mvn.confidence_region()
        ax[0].plot(test_x, mean,f"C{i}", label=generator.vocs.output_names[i])
        ax[0].fill_between(test_x, l[...,i].squeeze(), u[...,i].squeeze(), alpha=0.5)


    acq = generator.get_acquisition(model)(test_x.reshape(-1,1,1).double())

    ax[1].plot(test_x, acq, label='Acquisition Function')
    ax[1].legend()

In [ ]:
from emitopt.utils import get_valid_emittance_samples
beam_energy = 45*10**-3 # GeV 
q_len = 0.12 # m
distance = 1.33-0.265 #m

data = X.data

data["grad"] = data["AWA:Bira3Ctrl:Ch04"] * 100*8.93e-3
data["int_grad"] = data["grad"]*q_len*10

data["sx_m"] = 3.9232781168265036e-05 * data["13ARV1:Sx"]

data


In [ ]:
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.priors import GammaPrior
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from gpytorch import ExactMarginalLogLikelihood

from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll

train_x = torch.tensor(data.dropna()["int_grad"].to_numpy()).double().unsqueeze(1)
train_y = torch.tensor(data.dropna()["sx_m"].to_numpy()).double().unsqueeze(1)

print(train_x.shape)
print(train_y.shape)
input_transform = Normalize(1)
outcome_transform = Standardize(1)
covar_module = ScaleKernel(PolynomialKernel(power=2))
#covar_module = MaternKernel()

model = SingleTaskGP(train_x, 
                     train_y, 
                     input_transform=input_transform,
                     outcome_transform=outcome_transform, 
                     covar_module = covar_module
                     )

mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)


(emits_at_target_valid,
 emits_sq_at_target,
 is_valid,
 sample_validity_rate) = get_valid_emittance_samples(model, beam_energy,
                                                     q_len,
                                                     distance, n_samples=50, n_steps_quad_scan=10, visualize=True)

In [ ]:
plt.hist(emits_at_target_valid.flatten()*90)
plt.title('Distribution of Sampled Emittances')
plt.xlabel('Emittance')
plt.ylabel('Probability Density')